In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from joblib import dump

import logging
logging.basicConfig(
            format='%(asctime)s %(levelname)-8s %(message)s',
            level=logging.INFO,
            datefmt='%Y-%m-%d %H:%M:%S')

data = pd.read_csv("./data/used_car_train_20200313_revised.csv", index_col='SaleID')
X_test = pd.read_csv("./data/used_car_testA_20200313_revised.csv", index_col='SaleID')

def preprocess(X_train):
    '''
    transform 'notRepairedDamage column from 0, -, 1 to 0,1,2
    return transformed dataset
    '''
    new_col = X_train.notRepairedDamage.map(lambda x: 1 if x == '-' else int(float(x))*2)
    X_train = X_train.drop('notRepairedDamage',axis=1)
    X_train = X_train.join(new_col)
    
    return X_train

def validate(model, X_train, X_valid, y_train, y_valid):
    '''
    return validation scores of training data & validation data
    '''
    preds_valid = model.predict(X_valid)
    preds_train = model.predict(X_train)
    mae_valid = mean_absolute_error(preds_valid, y_valid)
    mae_train = mean_absolute_error(preds_train, y_train)
    print("Validation result:")
    print("train set mae on training set is {}".format(mae_train))
    print("valid set mae on validation set is {}".format(mae_valid))


# Preprocess data:
logging.info('Started...')
y = data['price']
X = data.drop('price',axis=1) #dropped about 15000 rows with missing values

full_cols = X.columns
selected_cols=['v_12','v_10','regDate','kilometer','v_0','v_14',
               'power','v_8','v_1','v_5','v_3','v_11',
               'v_9','v_6','v_4','notRepairedDamage','model',
               'v_2','v_13','name','brand','v_7','fuelType']

#split dataset & handling cat features
(X_train, X_valid, y_train, y_valid) = train_test_split(X, y, test_size=0.1)


2020-03-20 03:31:36 INFO     Started...


In [17]:
# date features preprocessing 
date_cols = ['regDate','creatDate']
X_train_date = X_train.copy()
X_test_date = X_test.copy()
def date_reformat(entry):
    entry = entry[0:4]+"-"+entry[4:6]+"-"+entry[6:]
    return pd.to_datetime(entry, format='%Y-%m-%d',errors='coerce')

def date_handle(X):
    for col in date_cols:
        X[col] = X[col].astype('str').map(date_reformat)
        return X

In [18]:
X_train_date = date_handle(X_train_date)
X_train_date.head()

,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
47871,96554,1998-11-09,1.0,0,1.0,0.0,0.0,75,15.0,0.0,...,0.247478,0.000000,0.188409,0.029221,0.049032,3.652875,-0.309587,-1.974535,-1.455930,0.408742
101491,16980,2002-06-02,0.0,0,0.0,0.0,0.0,105,15.0,0.0,...,0.258889,0.095431,0.117986,0.030695,0.063888,-3.322745,1.147760,-1.191454,-0.696659,0.439484
103198,191717,2011-09-04,8.0,0,2.0,1.0,1.0,143,15.0,0.0,...,0.252745,0.000449,0.114933,0.085430,0.062777,2.247665,-2.175466,1.695588,0.382135,1.223086
43032,67965,2009-08-10,74.0,25,3.0,1.0,0.0,109,9.0,0.0,...,0.249709,0.086348,0.027272,0.080753,0.109822,-3.475882,-0.121693,1.241136,2.128892,-0.300713
105100,36889,2002-06-11,48.0,14,1.0,1.0,0.0,75,15.0,0.0,...,0.237781,0.074794,0.115923,0.033357,0.088119,-1.218481,1.883437,-2.048145,0.763097,1.189195


In [19]:
X_test_date = date_handle(X_test_date)
X_test_date.head()

,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
150000,66932,2011-12-12,222.0,4,5.0,1.0,1.0,313,15.0,0.0,...,0.264405,0.121800,0.070899,0.106558,0.078867,-7.050969,-0.854626,4.800151,0.620011,-3.664654
150001,174960,1999-02-11,19.0,21,0.0,0.0,0.0,75,12.5,1.0,...,0.261745,0.000000,0.096733,0.013705,0.052383,3.679418,-0.729039,-3.796107,-1.541230,-0.757055
150002,5356,2009-03-04,82.0,21,0.0,0.0,0.0,109,7.0,0.0,...,0.260216,0.112081,0.078082,0.062078,0.050540,-4.926690,1.001106,0.826562,0.138226,0.754033
150003,50688,2010-04-05,0.0,0,0.0,0.0,1.0,160,7.0,0.0,...,0.260466,0.106727,0.081146,0.075971,0.048268,-4.864637,0.505493,1.870379,0.366038,1.312775
150004,161428,1997-07-03,26.0,14,2.0,0.0,0.0,75,15.0,0.0,...,0.250999,0.000000,0.077806,0.028600,0.081709,3.616475,-0.673236,-3.197685,-0.025678,-0.101290


In [20]:
for col in date_cols:
    print(X_train_date[col].value_counts(dropna=False, ascending=True))

2015-12-09        1
1991-08-01        1
1991-08-07        1
1991-12-11        1
1991-10-11        1
              ...  
2005-05-12       92
2003-03-11       97
2006-06-08       97
2003-03-10      102
NaT           10215
Name: regDate, Length: 3597, dtype: int64
20160115       1
20160130       1
20160131       1
20151217       1
20151227       1
            ... 
20160312    4858
20160402    4873
20160320    4890
20160404    5030
20160403    5267
Name: creatDate, Length: 95, dtype: int64


In [21]:
X_train_date[X_train_date['regDate'].isnull()]

,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,,,,,,,,,,,,,,,,,,,,,
14787,130710,NaT,29.0,0,1.0,0.0,0.0,0,15.0,0.0,...,0.236591,0.000000,0.134541,0.000943,0.096127,4.566499,0.752478,-5.104044,-0.242418,-0.341549
39153,36163,NaT,1.0,13,5.0,0.0,0.0,160,15.0,0.0,...,0.278894,0.000102,0.010069,0.063881,0.023589,2.349878,-2.567404,-0.586714,-0.775417,0.735307
120374,422,NaT,26.0,14,NaN,0.0,0.0,0,15.0,0.0,...,0.000000,0.113103,1.212394,0.051935,0.001277,2.961807,18.159437,4.484596,-0.241130,-0.142837
85211,325,NaT,10.0,9,1.0,0.0,0.0,58,15.0,-,...,0.233975,0.088571,0.073544,0.006039,0.108346,-1.393719,3.130423,-4.680589,1.245304,0.407558
86426,188186,NaT,26.0,14,0.0,0.0,0.0,75,15.0,0.0,...,0.256282,0.000000,0.053802,0.036917,0.075924,3.341369,-1.108247,-2.710125,0.062013,0.046401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37567,82316,NaT,69.0,6,2.0,0.0,0.0,85,15.0,0.0,...,0.254026,0.000000,0.071541,0.022634,0.067965,3.781680,-0.482364,-3.730952,-0.336485,0.279911
19782,87071,NaT,29.0,0,NaN,NaN,NaN,0,0.5,-,...,0.000000,0.000000,1.217424,0.005743,0.052175,11.605085,16.368552,0.371422,-0.536776,-0.983317
120830,80661,NaT,31.0,10,0.0,0.0,0.0,163,15.0,-,...,0.269070,0.000000,0.101964,0.058936,0.018317,2.594094,-1.961550,-0.117721,-1.589545,-0.490679


In [22]:
X_train[X_train_date['regDate'].isnull()]['regDate'].sort_values()

SaleID
29233     19910001
103842    19910001
9253      19910001
27475     19910001
118491    19910001
            ...   
121364    20150010
136211    20150010
132449    20150011
125727    20150011
128097    20150011
Name: regDate, Length: 10215, dtype: int64

In [23]:
for col in date_cols:
    print(X_test[col].value_counts(dropna=False, ascending=True))

19920801     1
19930905     1
19920804     1
20120007     1
19921208     1
            ..
20000005    50
20000009    53
20000012    54
20000011    56
20000004    60
Name: regDate, Length: 3835, dtype: int64
20160211       1
20150611       1
20160201       1
20160102       1
20151102       1
            ... 
20160314    1806
20160404    1810
20160321    1825
20160320    1858
20160403    1966
Name: creatDate, Length: 69, dtype: int64


In [24]:
X_test[X_test_date['regDate'].isnull()]['regDate'].sort_values()

SaleID
166359    19910001
186959    19910001
184460    19910001
164777    19910001
150858    19910001
            ...   
161728    20150010
183897    20150010
183950    20150011
167810    20150011
160651    20150011
Name: regDate, Length: 3754, dtype: int64

In [25]:
#Preprocessing

X_train = preprocess(X_train)
X_valid = preprocess(X_valid)
cols = X_train.columns

#Imputation
# num_imputer = SimpleImputer(strategy='mean') # not used, no numeric features imputed
cat_imputer = SimpleImputer(strategy='median')
catish_cols = X_train.loc[:,X_train.nunique().sort_values()<99000].columns
print(catish_cols)

X_train = pd.DataFrame(cat_imputer.fit_transform(X_train))
X_train.columns = cols
X_train[catish_cols]= X_train[catish_cols].astype('int64')

X_valid = pd.DataFrame(cat_imputer.transform(X_valid))
X_valid.columns = cols
X_valid[catish_cols]= X_valid[catish_cols].astype('int64')

X_train.drop(['offerType', 'seller', 'creatDate'], axis=1, inplace=True)
X_valid.drop(['offerType', 'seller', 'creatDate'], axis=1, inplace=True)

# After investigation, I decided to only consider year parts since it's reasonable in reality
# and I'm going to drop creatDate, due to it's weak coeff with price in Permutation Importance from rf_model

X_train['regDate'] = X_train['regDate'].map(lambda entry: str(entry)[0:4]).astype('int64')
X_valid['regDate'] = X_valid['regDate'].map(lambda entry: str(entry)[0:4]).astype('int64')

Index(['name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox',
       'power', 'kilometer', 'regionCode', 'seller', 'offerType', 'creatDate',
       'notRepairedDamage'],
      dtype='object')


In [26]:
#TEST
# X_test = preprocess(X_test)
# X_test = pd.DataFrame(cat_imputer.transform(X_test))
# X_test.columns = cols
# X_test[catish_cols]= X_test[catish_cols].astype('int64')
# X_test.drop(['offerType', 'seller', 'creatDate','name'], axis=1, inplace=True)
# X_test['regDate'] = X_test['regDate'].map(lambda entry: str(entry)[0:4])

In [28]:
%%time
#XGBoost
from xgboost import XGBRegressor
X_train = X_train[selected_cols]
X_valid = X_valid[selected_cols]
xgb_model = XGBRegressor(n_estimators=1000, random_state=42, n_jobs=-1, learn_rate=0.01)
xgb_model.fit(X_train, y_train, early_stopping_rounds=5, 
              eval_set=[(X_valid, y_valid)], eval_metric='mae')


[0]	validation_0-mae:4150.76514
Will train until validation_0-mae hasn't improved in 5 rounds.
[1]	validation_0-mae:2948.41016
[2]	validation_0-mae:2134.70508
[3]	validation_0-mae:1609.30298
[4]	validation_0-mae:1279.06909
[5]	validation_0-mae:1075.38452
[6]	validation_0-mae:957.61011
[7]	validation_0-mae:885.60028
[8]	validation_0-mae:845.30170
[9]	validation_0-mae:819.80292
[10]	validation_0-mae:803.64502
[11]	validation_0-mae:792.02527
[12]	validation_0-mae:782.95099
[13]	validation_0-mae:773.78046
[14]	validation_0-mae:767.92560
[15]	validation_0-mae:760.12744
[16]	validation_0-mae:756.78717
[17]	validation_0-mae:755.50153
[18]	validation_0-mae:752.74286
[19]	validation_0-mae:748.45605
[20]	validation_0-mae:745.50348
[21]	validation_0-mae:740.77930
[22]	validation_0-mae:736.01764
[23]	validation_0-mae:733.96735
[24]	validation_0-mae:731.52387
[25]	validation_0-mae:727.91773
[26]	validation_0-mae:724.69757
[27]	validation_0-mae:717.43219
[28]	validation_0-mae:716.51312
[29]	validati

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learn_rate=0.01, learning_rate=0.300000012, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method=None, validate_parameters=False, verbosity=None)

In [ ]:
# (X_train, y_train) = X_train[:10], y_train[:10]
# Randomized Grip Search for learning_rate and max_depth
# Tune HyperParameters
learning_rate = [float(x)/1000 for x in np.linspace(1,100, num=11)]
max_depth = [int(x) for x in np.linspace(10,110,num=11)]
max_depth.append(None)

random_grid = {'max_depth':max_depth,
              'learning_rate':learning_rate}
logging.info('Data preprocessing finished.\n')
model = XGBRegressor()
random_model = RandomizedSearchCV(estimator=model, param_distributions=random_grid, 
                                      n_iter=150, cv=5, verbose=2, random_state=42, 
                                      n_jobs=-1)
X_full = pd.concat([X_train, X_valid])
y_full = pd.concat([y_train, y_valid])
random_model.fit(X_full, y_full)


2020-03-20 03:41:37 INFO     Data preprocessing finished.



Fitting 5 folds for each of 132 candidates, totalling 660 fits


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 132 is smaller than n_iter=150. Running 132 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
